# Socio-economic Indicators

Notebook environment to migrate netcdf files to CF compliant zarr

In [1]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

<IPython.core.display.Javascript object>

### Configure OS independent paths

In [2]:
# Import standard packages
import os
import pathlib

import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import math

# Make root directories importable by appending root to path
cwd = pathlib.Path().resolve()
sys.path.append(os.path.dirname(cwd))
sys.path.append(r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\coclicodata')

# Get root paths
home = pathlib.Path().home()
root = home.root

# Import custom functionality
from etl import p_drive
from etl.CF_compliancy_checker import check_compliancy, save_compliancy

# Define (local and) remote drives
gca_data_dir = p_drive.joinpath("1000545-054-globalbeaches", "15_GlobalCoastalAtlas", "datasets")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    home.joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"Anaconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml"
    )
)

C:\Users\heijd_de\AppData\Local\Temp\ipykernel_2700\2707390607.py:7: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


<IPython.core.display.Javascript object>

In [3]:
# Project paths & files (manual input)
dataset_dir = gca_data_dir.joinpath("05_Worldpop")
dataset_dir_pop = dataset_dir.joinpath("worldpop.nc")
dataset_out_file = "worldpop"
CF_dir = gca_data_dir.joinpath(r"CF")  # directory to save output CF check files

<IPython.core.display.Javascript object>

In [4]:
# # # write csv to netcdf

# df_transects = pd.read_csv(r'P:\1000545-054-globalbeaches\03_Global_shorelines_update_2020\Transect_Mapping\Version_16102018\CSV\ShorelineMonitor_1984_2021_v1.5_set1_filtered.csv')
# gdf_transects = gpd.GeoDataFrame(df_transects, geometry=gpd.points_from_xy(df_transects.Intersect_lon, df_transects.Intersect_lat)).set_crs('epsg:4326').to_crs('+proj=wintri')
# print(len(gdf_transects))
# gdf_transects.head(2)

<IPython.core.display.Javascript object>

In [5]:
# df_socio = pd.read_csv(r'P:\11207765-jrc-dems-and-characteriza\Task_2\JRC_deliverable\3_Indicators\GCC_socioeconomic_filtered.csv')
# gdf_socio = gpd.GeoDataFrame(df_socio, geometry=gpd.points_from_xy(df_socio.lon, df_socio.lat)).set_crs('epsg:4326').to_crs('+proj=wintri')
# print(len(gdf_socio))
# gdf_socio.head(2)

<IPython.core.display.Javascript object>

In [6]:
# gdf_socio = gdf_transects.sjoin_nearest(gdf_socio, how = 'left', max_distance = 1000).dropna(subset = 'pop_1_m')

# gdf_socio.head(2)

<IPython.core.display.Javascript object>

In [7]:
# gdf_socio = gdf_socio[['transect_id', 'country_id', 'continent', 'country_name', 
#                         'Start_lon', 'Start_lat', 'Intersect_lon', 'Intersect_lat', 'End_lon',
#                         'End_lat', 'pop_1_m', 'pop_5_m', 'pop_10_m', 'pop_all', 'roads',
#                         'railways', 'ports', 'airports', 'gdp_ppp_usd2017_2015',
#                         'gdp_ppp_grid_2015']]

<IPython.core.display.Javascript object>

In [8]:
# # Convert the pandas dataframe to an xarray dataset
# ds = xr.Dataset.from_dataframe(gdf_socio)

# # Write the xarray dataset to a netCDF file
# ds.to_netcdf(dataset_dir_socio)

<IPython.core.display.Javascript object>

### Check CF compliancy original NetCDF files

In [9]:
# open datasets
ds_pop = xr.open_dataset(dataset_dir_pop)

# check original dataset
ds_pop 

<xarray.Dataset>
Dimensions:               (index: 1723302)
Coordinates:
  * index                 (index) int64 0 1 2 3 ... 1739823 1739824 1739825
Data variables: (12/20)
    transect_id           (index) object ...
    country_id            (index) object ...
    continent             (index) object ...
    country_name          (index) object ...
    Start_lon             (index) float64 ...
    Start_lat             (index) float64 ...
    ...                    ...
    roads                 (index) float64 ...
    railways              (index) float64 ...
    ports                 (index) float64 ...
    airports              (index) float64 ...
    gdp_ppp_usd2017_2015  (index) float64 ...
    gdp_ppp_grid_2015     (index) float64 ...

<IPython.core.display.Javascript object>

In [10]:
drop_vars = ['roads', 'railways', 'ports', 'airports', 'gdp_ppp_usd2017_2015', 'gdp_ppp_grid_2015']
ds_pop  = ds_pop.drop(drop_vars)

<IPython.core.display.Javascript object>

In [11]:
ds_pop

<xarray.Dataset>
Dimensions:        (index: 1723302)
Coordinates:
  * index          (index) int64 0 1 2 3 4 ... 1739822 1739823 1739824 1739825
Data variables: (12/14)
    transect_id    (index) object ...
    country_id     (index) object ...
    continent      (index) object ...
    country_name   (index) object ...
    Start_lon      (index) float64 ...
    Start_lat      (index) float64 ...
    ...             ...
    End_lon        (index) float64 ...
    End_lat        (index) float64 ...
    pop_1_m        (index) float64 ...
    pop_5_m        (index) float64 ...
    pop_10_m       (index) float64 ...
    pop_all        (index) float64 ...

<IPython.core.display.Javascript object>

In [12]:
%%capture cap --no-stderr
# check original CF compliancy

check_compliancy(testfile= dataset_dir_pop, 
                 working_dir=CF_dir
                 )


<IPython.core.display.Javascript object>

In [13]:
# save original CF compliancy
save_compliancy(cap, testfile=dataset_dir_pop, working_dir=CF_dir)

{'P:\\1000545-054-globalbeaches\\15_GlobalCoastalAtlas\\datasets\\05_Worldpop\\worldpop.nc': {'warnings': '27', 'errors': '4'}}


<IPython.core.display.Javascript object>

### Make CF compliant alterations to the NetCDF files (dataset dependent)

In [14]:
import json

# NetCDF attribute alterations
f_global = open(r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\datasets\05_Worldpop\metadata_worldpop.json')
meta_global = json.load(f_global)

for attr_name, attr_val in meta_global.items():
    if attr_name == 'PROVIDERS':
        attr_val = json.dumps(attr_val)
    ds_pop.attrs[attr_name] = attr_val

ds_pop.attrs['Conventions'] = "CF-1.8"

<IPython.core.display.Javascript object>

In [15]:
ds_pop

<xarray.Dataset>
Dimensions:        (index: 1723302)
Coordinates:
  * index          (index) int64 0 1 2 3 4 ... 1739822 1739823 1739824 1739825
Data variables: (12/14)
    transect_id    (index) object ...
    country_id     (index) object ...
    continent      (index) object ...
    country_name   (index) object ...
    Start_lon      (index) float64 ...
    Start_lat      (index) float64 ...
    ...             ...
    End_lon        (index) float64 ...
    End_lat        (index) float64 ...
    pop_1_m        (index) float64 ...
    pop_5_m        (index) float64 ...
    pop_10_m       (index) float64 ...
    pop_all        (index) float64 ...
Attributes: (12/21)
    TITLE:               World Population
    TITLE_ABBREVIATION:  world_pop
    DESCRIPTION:         The World Population dataset provides a global popul...
    SHORT_DESCRIPTION:   The World Population dataset provides a global popul...
    INSTITUTION:         University of Southampton
    PROVIDERS:           {"name": "Deltares", "url": "www.southampton.ac.uk",...
    ...                  ...
    DOI:                 doi:10.5258/SOTON/WP00685
    LONG_NAME:           WORLD_POPULATION
    UNITS:               
    COMMENT:             This dataset is modified from the original Bondarenk...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [16]:
# combine start and end coordinates into a transect
from shapely.geometry import LineString

start_lons = ds_pop["Start_lon"].values
start_lats = ds_pop["Start_lat"].values
end_lons = ds_pop["End_lon"].values
end_lats = ds_pop["End_lat"].values
coords = zip(zip(start_lons, start_lats), zip(end_lons, end_lats))

ds_pop["transect_geom"] = (
    ["index"],
    [str(LineString(line)) for line in coords],
)
ds_pop["transect_geom"].attrs["long_name"] = "Transect Geometry"

<IPython.core.display.Javascript object>

In [17]:
keep_vars = ['transect_id', 'country_name', 'continent', 'Start_lon', 'Start_lat', 'transect_geom', 'pop_1_m', 'pop_5_m', 'pop_10_m', 'pop_all']
allvars = list(ds_pop.keys())
delete_vars = list(set(allvars).difference(set(keep_vars)))

ds_pop = ds_pop.drop(delete_vars)

<IPython.core.display.Javascript object>

In [18]:
ds_pop

<xarray.Dataset>
Dimensions:        (index: 1723302)
Coordinates:
  * index          (index) int64 0 1 2 3 4 ... 1739822 1739823 1739824 1739825
Data variables:
    transect_id    (index) object ...
    continent      (index) object ...
    country_name   (index) object ...
    Start_lon      (index) float64 -74.39 -74.38 -74.38 ... 39.94 39.94 39.94
    Start_lat      (index) float64 -50.38 -50.38 -50.38 ... 64.7 64.7 64.7
    pop_1_m        (index) float64 ...
    pop_5_m        (index) float64 ...
    pop_10_m       (index) float64 ...
    pop_all        (index) float64 ...
    transect_geom  (index) <U92 'LINESTRING (-74.3863095545 -50.3776589451, -...
Attributes: (12/21)
    TITLE:               World Population
    TITLE_ABBREVIATION:  world_pop
    DESCRIPTION:         The World Population dataset provides a global popul...
    SHORT_DESCRIPTION:   The World Population dataset provides a global popul...
    INSTITUTION:         University of Southampton
    PROVIDERS:           {"name": "Deltares", "url": "www.southampton.ac.uk",...
    ...                  ...
    DOI:                 doi:10.5258/SOTON/WP00685
    LONG_NAME:           WORLD_POPULATION
    UNITS:               
    COMMENT:             This dataset is modified from the original Bondarenk...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [19]:
# NetCDF variable and dimension alterations

# rename or swap dimension names, the latter in case the name already exists as coordinate
ds_pop = ds_pop.rename_dims({"index": "stations"})

f_vars = open(r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\datasets\05_Worldpop\vars_worldpop.json')
meta_vars = json.load(f_vars)

for var_name, var_dict in meta_vars.items():
    ds_pop = ds_pop.rename_vars({var_name : var_dict['name']})
    for key, value in var_dict.items():
        if key != 'name':
            ds_pop[var_dict['name']].attrs[key] = value


# change dtypes
object_vars = ['transect_id', 'country', 'continent', 'transect_geom']
for ov in object_vars:
    ds_pop[ov] = ds_pop[ov].astype('S')

all_vars = list(ds_pop.keys())
data_vars = ['pop_1_m', 'pop_5_m', 'pop_10_m', 'pop_tot']
# set some data variables to coordinates to avoid duplication of dimensions in later stage
ds_pop = ds_pop.set_coords([v for v in all_vars if v not in data_vars])


# drop index xarray
ds_pop = ds_pop.drop('index')

<IPython.core.display.Javascript object>

In [20]:
# check the xarray dataset, best practice is to have as many as possible bold dimensions (dimension == coordinate name).
# in this way, the Front-End can access the variable directly without having to index the variable first
# dataset["scenarios"]
ds_pop

<xarray.Dataset>
Dimensions:        (stations: 1723302)
Coordinates:
    transect_id    (stations) |S15 b'BOX_028_183_0' ... b'BOX_211_067_153'
    continent      (stations) |S23 b'South America' ... b'Europe'
    country        (stations) |S40 b'Chile' b'Chile' ... b'Russia' b'Russia'
    lon            (stations) float64 -74.39 -74.38 -74.38 ... 39.94 39.94 39.94
    lat            (stations) float64 -50.38 -50.38 -50.38 ... 64.7 64.7 64.7
    transect_geom  (stations) |S92 b'LINESTRING (-74.3863095545 -50.377658945...
Dimensions without coordinates: stations
Data variables:
    pop_1_m        (stations) float64 ...
    pop_5_m        (stations) float64 ...
    pop_10_m       (stations) float64 ...
    pop_tot        (stations) float64 ...
Attributes: (12/21)
    TITLE:               World Population
    TITLE_ABBREVIATION:  world_pop
    DESCRIPTION:         The World Population dataset provides a global popul...
    SHORT_DESCRIPTION:   The World Population dataset provides a global popul...
    INSTITUTION:         University of Southampton
    PROVIDERS:           {"name": "Deltares", "url": "www.southampton.ac.uk",...
    ...                  ...
    DOI:                 doi:10.5258/SOTON/WP00685
    LONG_NAME:           WORLD_POPULATION
    UNITS:               
    COMMENT:             This dataset is modified from the original Bondarenk...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [21]:
# save new .nc files
ds_pop.to_netcdf(path=str(dataset_dir_pop).replace(".nc", "_CF.nc"))

<IPython.core.display.Javascript object>

### Check CF compliancy altered NetCDF files

In [22]:
%%capture cap --no-stderr
# check altered CF compliancy

check_compliancy(testfile=str(dataset_dir_pop).replace(".nc", "_CF.nc"), working_dir=CF_dir)

<IPython.core.display.Javascript object>

In [23]:
# save altered CF compliancy
save_compliancy(
    cap,
    testfile=str(dataset_dir_pop).replace(".nc", "_CF.nc"),
    working_dir=CF_dir,
)

{'P:\\1000545-054-globalbeaches\\15_GlobalCoastalAtlas\\datasets\\05_Worldpop\\worldpop_CF.nc': {'warnings': '0', 'errors': '0'}}


<IPython.core.display.Javascript object>

### write data to Zarr files

In [24]:
# export to zarr in write mode (to overwrite if exists)
ds_pop.to_zarr(dataset_dir.joinpath("%s.zarr" % dataset_out_file), mode="w")

<IPython.core.display.Javascript object>

In [25]:
test = xr.open_zarr(dataset_dir.joinpath("%s.zarr" % dataset_out_file))

<IPython.core.display.Javascript object>